In [ ]:
# Imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
# For reproducibility
# 매 실험마다 같은 결과값을 도출해내기위해 시드값을 고정
torch.manual_seed(1)

In [ ]:
# Training Data
# x_data 는 각각의 학생들이 강의를 시청한 시간과 code lab 에서 보낸 시간을 의미한다.
# y_data 는 그에 따른 학생들의 course 통과여부를 의미한다.
x_data = [[1, 2],
          [2, 3],
          [3, 1],
          [4, 3],
          [5, 3],
          [6, 2]]
y_data = [[0],
          [0],
          [0],
          [1],
          [1],
          [1]]

# 이 데이터들을 torch.Tensor 로 변환
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

# 데이터의 shape 확인
print(x_train.shape)
print(y_train.shape)

In [ ]:
# Computing the Hypothesis

# exponential function 확인
print('e^1 equals: ', torch.exp(torch.FloatTensor([1])))

# torch.exp() 를 이용하여 hypothesis function 구현
W = torch.zeros((2, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

hypothesis = 1/(1 + torch.exp(-(x_train.matmul(W) + b)))

print(hypothesis)     # 현재 W 와 bias 의 모든 element 는 0 이므로 0.5 를 출력한다.
print(hypothesis.shape)   # x_train 과 W 의 shape 가 각각  6*2, 2*1 이므로 6*1 의 shape 를 가진다.

In [ ]:
# More simpler method, torch.sigmoid()

# Sigmoid function 확인
print('1/(1+e^{-1}) equals: ', torch.sigmoid(torch.FloatTensor([1])))

# torch.sigmoid() 를 이용하여 더 간단한 hypothesis function 구현
hypothesis = torch.sigmoid(x_train.matmul(W) + b)
print(hypothesis)
print(hypothesis.shape)

In [ ]:
#Computing the Cost Function (Low-level)

print(hypothesis)
print(y_train)

# 한 개의 원소에 대해 loss 는 다음과 같이 계산된다.
-(y_train[0] * torch.log(hypothesis[0]) +
  (1 - y_train[0]) * torch.log(1 - hypothesis[0]))
# hypothesis[0] 이 1 일 확률이라면 (1 - hypothesis) 는 0 일 확률이다.

# 전체 batch 에 대해 losses 를 계산하면 다음과 같다.
losses = -(y_train * torch.log(hypothesis) +
           (1 - y_train) * torch.log(1 - hypothesis))
print(losses)   # 6 * 1

# .mean() 을 이용하여 각각의 losses 의 평균을 구한다.
cost = losses.mean()
print(cost)   # scalar

In [ ]:
# Computing the Cost function with F.binary_cross_entropy
# 위의 과정을 모두 포함하여 cost 를 구하는 method F.binary_cross_entropy 가 있다.

F.binary_cross_entropy(hypothesis, y_train)   # equals: cost

In [ ]:
# Training with Low-Level Binary Cross Entropy Loss

x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]]
y_data = [[0], [0], [0], [1], [1],[1]]
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

In [ ]:
# 모델 초기화
W = torch.zeros((2, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

# optimizer 설정
optimizer = optim.SGD([W, b], lr=1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
  # Cost 계산
  hypothesis = torch.sigmoid(x_train.matmul(W) + b)
  cost = -(y_train * torch.log(hypothesis) +
          (1 - y_train) * torch.log(1 - hypothesis)).mean()

  # Cost 로 H(x) 개선
  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  # 100번 마다 로그 출력
  if epoch % 100 == 0:
    print('Epoch {:4d}/{} Cost: {:.6f}'.format(
        epoch, nb_epochs, cost.item()
    ))

In [ ]:
# Loading Real Data
# 실제 데이터를 load 함

from google.colab import drive
drive.mount('/content/drive')

import numpy as np

xy = np.loadtxt('/content/drive/MyDrive/AI_class/Deep_learning_class/data-03-diabetes.csv', delimiter=',', dtype=np.float32)
x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

print(x_train[0:5])   # 5 개의 데이터만 출력
print(y_train[0:5])

In [ ]:
# Training with Real Data using low-level Binary Cross Entropy Loss
# 모델 초기화
W = torch.zeros((8, 1), requires_grad=True)     # 위에서 load 한 실제 데이터의 dimension 이 8 이므로 W 는 8 * 1 의 shape 를 가진다.
b = torch.zeros(1, requires_grad=True)

# optimizer 설정
optimizer = optim.SGD([W, b], lr=1)

nb_epochs = 100
for epoch in range(nb_epochs + 1):

  # Cost 계산
  hypothesis = torch.sigmoid(x_train.matmul(W) + b)
  cost = -(y_train * torch.log(hypothesis) + (1 - y_train) * torch.log(1 - hypothesis)).mean()

  # cost 로 H(x) 개선
  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  # 10 번 마다 로그 출력
  if epoch % 10 == 0:
    print('Epoch {:4d}/{} Cost: {:.6f}'.format(
        epoch, nb_epochs, cost.item()
    ))

In [ ]:
# Training with Real Data using F.binary_cross_entropy
# 모델 초기화
W = torch.zeros((8, 1), requires_grad=True)     # 위에서 load 한 실제 데이터의 dimension 이 8 이므로 W 는 8 * 1 의 shape 를 가진다.
b = torch.zeros(1, requires_grad=True)

# optimizer 설정
optimizer = optim.SGD([W, b], lr=1)

nb_epochs = 100
for epoch in range(nb_epochs + 1):

  # Cost 계산
  hypothesis = torch.sigmoid(x_train.matmul(W) + b)
  cost = F.binary_cross_entropy(hypothesis, y_train)

  # cost 로 H(x) 개선
  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  # 10 번 마다 로그 출력
  if epoch % 10 == 0:
    print('Epoch {:4d}/{} Cost: {:.6f}'.format(
        epoch, nb_epochs, cost.item()
    ))

In [ ]:
# Checking the Accuracy our Model

# hypothesis 의 Tensor 를 출력해보면 다음과 같다.
hypothesis = torch.sigmoid(x_train.matmul(W) + b)
print(hypothesis[:5])   # 처음부터 5개의 데이터를 출력

In [ ]:
# 출력된 hypothesis 를 binary predictions 으로 바꾼다.(0.5 를 기준으로)
prediction = (hypothesis >= torch.FloatTensor([0.5])) # Boolean equation 이므로 prediction 에 저장된 값은 True or False 이다.
print(prediction[:5].type(torch.uint8))

In [ ]:
# correct_prsdiction 에 이 값을 y_train 값과 비교하여 일치하면 1, 일치하지 않으면 0 을 저장한다.
print(prediction[:5].type(torch.uint8))
print(y_train[:5])

correct_prediction = prediction.float() == y_train
print(correct_prediction[:5].type(torch.uint8))



In [ ]:
# 모델의 정확도 출력
accuracy = correct_prediction.sum().item()/len(correct_prediction)
print('The model has an accuracy of {:2.2f}% for the training set.'.format(accuracy * 100))
# 백분율로 나타냈으므로 accuracy 에 100 을 곱하여 출력했다.

In [ ]:
# High-level Implementation with nn.Module

class BinaryClassifier(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear = nn.Linear(8, 1)
    self.sigmoid = nn.Sigmoid()

  def forward(self, x):
    return self.sigmoid(self.linear(x))

model = BinaryClassifier()

# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=1)

nb_epochs = 100
for epoch in range(nb_epochs + 1):

  # H(x) 계산
  hypothesis = model(x_train)

  # Cost 계산
  cost = F.binary_cross_entropy(hypothesis, y_train)

  # cost로 H(x) 개선
  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  # 10 번 마다 로그 출력
  if epoch % 10 == 0:
    prediction = hypothesis >= torch.FloatTensor([0.5])
    correct_prediction = prediction.float() == y_train
    accuracy = correct_prediction.sum().item() / len(correct_prediction)
    print('Epoch {:4d}/{} Cost: {:.6f} Accuracy {:2.2f}%'.format(
        epoch, nb_epochs, cost.item(), accuracy * 100,
    ))